Here we use a subset of the dataset provided by Kaggle's competition [M5 Forecasting - Accuracy
Estimate the unit sales of Walmart retail goods](https://www.kaggle.com/c/m5-forecasting-accuracy).

We start with the data provided in the files:

* calendar.csv - Contains information about the dates on which the products are sold.
* sell_prices.csv - Contains information about the price of the products sold per store and date.
* sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

And we perform some filtering in order to have the sample that we will be using for our project.

The data mentioned can be downloaded at: https://www.kaggle.com/c/m5-forecasting-accuracy/data

And the result after filtering can be downloaded from: https://github.com/dpbac/Forecasting-Walmart-sales-with-Azure/blob/master/data/walmart_tx_stores_10_items_with_day.csv

In [1]:
# Use it to decrease memory use

import numpy as np

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [2]:
import pandas as pd
import gc
import warnings 

pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

# Load Data

## Sales info

In [3]:
#  Includes sales [d_1 - d_1941]
df_walmart = pd.read_csv("./data/sales_train_evaluation.csv")
df_walmart = reduce_mem_usage(df_walmart)
df_walmart.head()

Mem. usage decreased to 96.13 Mb (78.8% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [4]:
df_walmart.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1317), int8(624), object(6)
memory usage: 96.1+ MB


This data cover stores in 3 states:

In [5]:
for state in df_walmart['state_id'].value_counts().index:
    print(state+" has "+ str(len(df_walmart[df_walmart['state_id']==state]['store_id'].unique()))+" stores.")

CA has 4 stores.
WI has 3 stores.
TX has 3 stores.


In [6]:
for state in df_walmart['state_id'].value_counts().index:
    print(state)
    for store in df_walmart[df_walmart['state_id']==state]['store_id'].unique():
        print("Number of items in store "+store+" :"+str(len(df_walmart[df_walmart['store_id']==store]['item_id'].unique())))

CA
Number of items in store CA_1 :3049
Number of items in store CA_2 :3049
Number of items in store CA_3 :3049
Number of items in store CA_4 :3049
WI
Number of items in store WI_1 :3049
Number of items in store WI_2 :3049
Number of items in store WI_3 :3049
TX
Number of items in store TX_1 :3049
Number of items in store TX_2 :3049
Number of items in store TX_3 :3049


Each store has 3049 items.

In [7]:
for state in df_walmart['state_id'].value_counts().index:
    print(state)
    for store in df_walmart[df_walmart['state_id']==state]['store_id'].unique():
        print("Number of departments in store "+store+" :"+str(len(df_walmart[df_walmart['store_id']==store]['dept_id'].unique())))

CA
Number of departments in store CA_1 :7
Number of departments in store CA_2 :7
Number of departments in store CA_3 :7
Number of departments in store CA_4 :7
WI
Number of departments in store WI_1 :7
Number of departments in store WI_2 :7
Number of departments in store WI_3 :7
TX
Number of departments in store TX_1 :7
Number of departments in store TX_2 :7
Number of departments in store TX_3 :7


In [8]:
df_walmart['dept_id'].unique()

array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3'], dtype=object)

In [9]:
df_walmart.groupby(['state_id','store_id','cat_id','dept_id']).count()

id  item_id  d_1  d_2  d_3  d_4  \
state_id store_id cat_id    dept_id                                         
CA       CA_1     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
         CA_2     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
         CA_3     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
         CA_4     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
TX       TX_1     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
         TX_2     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
         TX_3     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2      398      398  398  398  398  398   
                            FOODS_3      823      823  823  823  823  823   
                  HOBBIES   HOBBIES_1    416      416  416  416  416  416   
                            HOBBIES_2    149      149  149  149  149  149   
                  HOUSEHOLD HOUSEHOLD_1  532      532  532  532  532  532   
                            HOUSEHOLD_2  515      515  515  515  515  515   
WI       WI_1     FOODS     FOODS_1      216      216  216  216  216  216   
                            FOODS_2   

Let's consider the state of Texas (`TX`) and department `HOBBIES_2`

In [10]:
# Check number of time series and lengths
print("Number of stores in TX is {}.".format(len(df_walmart['store_id'][df_walmart['state_id']=='TX'].unique())))
print("Number of items in each store is {}.".format(len(df_walmart['item_id'][(df_walmart['state_id']=='TX')&(df_walmart['dept_id']=='HOBBIES_2')].unique())))
print("In all {} stores there are {} items.".format(len(df_walmart['store_id'][df_walmart['state_id']=='TX'].unique()),len(df_walmart['item_id'][(df_walmart['state_id']=='TX')&(df_walmart['dept_id']=='HOBBIES_2')].unique())*len(df_walmart['store_id'][df_walmart['state_id']=='TX'].unique())))

Number of stores in TX is 3.
Number of items in each store is 149.
In all 3 stores there are 447 items.


In [11]:
selected_articles = df_walmart['item_id'][(df_walmart['state_id']=='TX')&(df_walmart['dept_id']=='HOBBIES_2')][:10].values.tolist()
selected_articles

['HOBBIES_2_001',
 'HOBBIES_2_002',
 'HOBBIES_2_003',
 'HOBBIES_2_004',
 'HOBBIES_2_005',
 'HOBBIES_2_006',
 'HOBBIES_2_007',
 'HOBBIES_2_008',
 'HOBBIES_2_009',
 'HOBBIES_2_010']

In [12]:
# 10 items for each store in Texas
df_walmart_TX_HOBBIES_2 = df_walmart[(df_walmart['state_id']=='TX')&(df_walmart['item_id'].isin(selected_articles))]
del df_walmart
df_walmart_TX_HOBBIES_2.reset_index(drop=True, inplace=True)
df_walmart_TX_HOBBIES_2 = reduce_mem_usage(df_walmart_TX_HOBBIES_2)

Mem. usage decreased to  0.06 Mb (39.8% reduction)


In [13]:
df_walmart_TX_HOBBIES_2.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,0,0,0,0,...,1,1,1,1,1,1,3,0,0,1
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,1,0,0,1,0,0,0,4
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
df_walmart_TX_HOBBIES_2.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Columns: 1947 entries, id to d_1941
dtypes: int8(1941), object(6)
memory usage: 58.4+ KB


In [15]:
df_walmart_TX_HOBBIES_2.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.200000,0.333333,0.233333,0.133333,0.100000,0.100000,0.366667,0.400000,0.066667,0.166667,...,0.500000,0.366667,0.233333,0.600000,0.500000,0.300000,0.333333,0.366667,0.233333,0.433333
std,0.761124,0.922266,0.678911,0.345746,0.305129,0.402578,1.299425,1.162637,0.253708,0.592093,...,0.682288,0.668675,0.430183,0.932183,0.900192,0.595963,0.711159,1.159171,0.504007,0.858360
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.750000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,4.000000,4.000000,3.000000,1.000000,1.000000,2.000000,7.000000,5.000000,1.000000,3.000000,...,2.000000,2.000000,1.000000,4.000000,4.000000,2.000000,3.000000,6.000000,2.000000,4.000000


In [16]:
df_walmart_TX_HOBBIES_2 = pd.melt(df_walmart_TX_HOBBIES_2,
                   id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                   var_name = 'day',
                   value_name = 'demand')
df_walmart_TX_HOBBIES_2 = reduce_mem_usage(df_walmart_TX_HOBBIES_2)

Mem. usage decreased to  3.17 Mb (0.0% reduction)


In [17]:
df_walmart_TX_HOBBIES_2.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0


In [18]:
df_walmart_TX_HOBBIES_2.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58230 entries, 0 to 58229
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        58230 non-null  object
 1   item_id   58230 non-null  object
 2   dept_id   58230 non-null  object
 3   cat_id    58230 non-null  object
 4   store_id  58230 non-null  object
 5   state_id  58230 non-null  object
 6   day       58230 non-null  object
 7   demand    58230 non-null  int8  
dtypes: int8(1), object(7)
memory usage: 3.2+ MB


In [19]:
df_walmart_TX_HOBBIES_2['store_id'].value_counts()

TX_1    19410
TX_2    19410
TX_3    19410
Name: store_id, dtype: int64

In [20]:
df_walmart_TX_HOBBIES_2['item_id'].value_counts()

HOBBIES_2_003    5823
HOBBIES_2_008    5823
HOBBIES_2_009    5823
HOBBIES_2_007    5823
HOBBIES_2_005    5823
HOBBIES_2_001    5823
HOBBIES_2_010    5823
HOBBIES_2_004    5823
HOBBIES_2_002    5823
HOBBIES_2_006    5823
Name: item_id, dtype: int64

So our goal will be to forecast daily sales for the next `28 days` of item in `HOBBIES_2` department for the stores of Walmart in Texas.

We need now to filter also the other 2 datasets to include only information about those items.

## Calendar info

In [24]:
calendar = pd.read_csv("./data/calendar.csv", parse_dates=['date'])
calendar = reduce_mem_usage(calendar)
calendar.head()

Mem. usage decreased to  0.12 Mb (41.9% reduction)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [25]:
# removing columns related to CA and WI

calendar.drop(columns=['snap_CA', 'snap_WI'], inplace=True)

In [26]:
calendar.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int16         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int8          
 4   month         1969 non-null   int8          
 5   year          1969 non-null   int16         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_TX       1969 non-null   int8          
dtypes: datetime64[ns](1), int16(2), int8(3), object(6)
memory usage: 121.3+ KB


In [27]:
calendar['date'].min(),calendar['date'].max()

(Timestamp('2011-01-29 00:00:00'), Timestamp('2016-06-19 00:00:00'))

## Sell prices info

In [28]:
sell_prices = pd.read_csv('./data/sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
sell_prices.head()

Mem. usage decreased to 130.48 Mb (37.5% reduction)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [29]:
sell_prices = sell_prices[(sell_prices['store_id'].str.contains('TX'))&(sell_prices['item_id'].isin(selected_articles))]
sell_prices.reset_index(drop=True,inplace=True)

In [30]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,TX_1,HOBBIES_2_001,11102,5.46875
1,TX_1,HOBBIES_2_001,11103,5.46875
2,TX_1,HOBBIES_2_001,11104,5.46875
3,TX_1,HOBBIES_2_001,11105,5.46875
4,TX_1,HOBBIES_2_001,11106,5.46875


In [31]:
sell_prices.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7704 entries, 0 to 7703
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   store_id    7704 non-null   object 
 1   item_id     7704 non-null   object 
 2   wm_yr_wk    7704 non-null   int16  
 3   sell_price  7704 non-null   float16
dtypes: float16(1), int16(1), object(2)
memory usage: 150.6+ KB


In [32]:
sell_prices['store_id'].unique()

array(['TX_1', 'TX_2', 'TX_3'], dtype=object)

In [33]:
for store in sell_prices['store_id'].unique():
    print("Store {} has {} items".format(store,len(sell_prices['item_id'][sell_prices['store_id']=='TX_1'].unique())))

Store TX_1 has 10 items
Store TX_2 has 10 items
Store TX_3 has 10 items


In [34]:
# Uncomment if you want to save info in csv this partial data

# df_walmart_TX_HOBBIES_2.to_csv("../data/walmart/walmart_TX_HOBBIES_2.csv", index=False)
# calendar.to_csv("../data/walmart/calendar_reduced.csv", index=False)
# sell_prices.to_csv("../data/walmart/sell_prices_reduced.csv", index=False)

# Merge all information

In [35]:
# mergin calendar information
calendar.drop(['weekday', 'wday', 'month', 'year'], inplace = True, axis = 1)
data = pd.merge(df_walmart_TX_HOBBIES_2, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d'], inplace = True, axis = 1)
del df_walmart_TX_HOBBIES_2

In [36]:
data = reduce_mem_usage(data)

Mem. usage decreased to  6.00 Mb (0.0% reduction)


In [37]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0


In [38]:
# merge price info
data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
del calendar,sell_prices
gc.collect()

15

In [39]:
data = reduce_mem_usage(data)

Mem. usage decreased to  6.11 Mb (0.0% reduction)


In [40]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX,sell_price
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,NaN
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,1.969727
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,NaN
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,NaN
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,NaN


In [41]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58230 entries, 0 to 58229
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            58230 non-null  object        
 1   item_id       58230 non-null  object        
 2   dept_id       58230 non-null  object        
 3   cat_id        58230 non-null  object        
 4   store_id      58230 non-null  object        
 5   state_id      58230 non-null  object        
 6   day           58230 non-null  object        
 7   demand        58230 non-null  int8          
 8   date          58230 non-null  datetime64[ns]
 9   wm_yr_wk      58230 non-null  int16         
 10  event_name_1  4740 non-null   object        
 11  event_type_1  4740 non-null   object        
 12  event_name_2  120 non-null    object        
 13  event_type_2  120 non-null    object        
 14  snap_TX       58230 non-null  int8          
 15  sell_price    52938 non-null  float1

In [42]:
# Save reduced data

data.to_csv("./data/walmart_tx_stores_10_items_with_day.csv", index = False)